In [62]:
import boto3
import botocore
from pprint import pprint

# Create session
client = boto3.session.Session(profile_name="sbx").client(service_name='ec2', region_name='us-east-1')


## We are are looking are meta object in boto3

In [15]:
def using_meta_boject():
    """This function is describe all available regions using meta object
    Parameter: None
    Return: None
    """
    regions = []
    resource = boto3.session.Session(profile_name="sbx").resource(service_name='ec2', region_name='us-east-1')
    for each_region in resource.meta.client.describe_regions().get('Regions'):
        regions.append(each_region['RegionName'])
    print(len(regions), regions)  
    
using_meta_boject()    

17 ['eu-north-1', 'ap-south-1', 'eu-west-3', 'eu-west-2', 'eu-west-1', 'ap-northeast-3', 'ap-northeast-2', 'ap-northeast-1', 'sa-east-1', 'ca-central-1', 'ap-southeast-1', 'ap-southeast-2', 'eu-central-1', 'us-east-1', 'us-east-2', 'us-west-1', 'us-west-2']


### Working with collections. resource


## using resource object

In [17]:
## we .all(), filter(), limit()


In [59]:
def ec2_instance_id():
    ids = []
    resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')

    for all_ids in resource.instances.all():
        ids.append(all_ids.id)
    return ids    
    
    
ids = ec2_instance_id()  

print(ids)

['i-051dc527966309583', 'i-02386829b4ff948e7', 'i-06cce8b354391a48e', 'i-0becc932d948071d9', 'i-0944666a8b7b73b54', 'i-00f47179781a6bd19']


In [32]:
def ec2_instance_with_limit():
    ids = []
    resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')
    for all_ids in resource.instances.limit(1):
        ids.append(all_ids.id)
    print(ids)    
    
    
    
ec2_instance_with_limit()   

['i-051dc527966309583']


In [38]:
def ec2_instance_with_filter():
    ids = []
    Filter_running = {'Name': 'instance-state-name','Values': ['running']}
    resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')
    for each_instance in resource.instances.filter(Filters=[Filter_running]):
        print(each_instance.id)
    return None


ec2_instance_with_filter()
    

i-051dc527966309583
i-0becc932d948071d9


In [41]:
def ec2_instance_with_filter_tags():
    ids = []
    Filter_tags = {'Name': 'tag:Name','Values': ['python-boto3']}
    resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')
    for each_instance in resource.instances.filter(Filters=[Filter_tags]):
        print(each_instance.id)
    
    return None


ec2_instance_with_filter_tags()

i-02386829b4ff948e7
i-06cce8b354391a48e
i-0becc932d948071d9


## Using Waiters  to stop all instance  

In [52]:
def stop_all_instance():
    """Using resource object to stop all instances. 
    Parameter: None
    Return: None
    """
    client = boto3.session.Session(profile_name="shared").client(service_name='ec2', region_name='us-east-1')
    
    # creating waiter
    waiter = client.get_waiter('instance_stopped')
    resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')
    print("stopping all instances....")
    resource.instances.stop()
    waiter.wait(InstanceIds=ids)
    print("all instances stopped!!...")
  
stop_all_instance()
    
    
    

stopping all instances....
all instances stopped!!...


## Using Waiters to start all instance

In [55]:
 def start_all_instance():
   """Using resource object to start all instances. 
    Parameter: None
    Return: None
    """
    client = boto3.session.Session(profile_name="shared").client(service_name='ec2', region_name='us-east-1')
    
    # creating waiter
    waiter = client.get_waiter('instance_running')
    resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')
    print("starting all instances....")
    resource.instances.start()
    waiter.wait(InstanceIds=ids)
    print("all instances running!!...")
  
start_all_instance()

starting all instances....
all instances running!!...


### Using Waiters to terminate all instance

In [58]:
def terminate_all_instances():
    """Using resource object to terminate all instances. 
    Parameter: None
    Return: None
    """
    client = boto3.session.Session(profile_name="shared").client(service_name='ec2', region_name='us-east-1')
    
    # creating waiter
    waiter = client.get_waiter('instance_terminated')
    resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')
    print("terminating all instances....")
    resource.instances.terminate()
    waiter.wait(InstanceIds=ids)
    print("all instances terminated")
  


In [60]:
stop_all_instance()

stopping all instances....


ClientError: An error occurred (IncorrectInstanceState) when calling the StopInstances operation: This instance 'i-0becc932d948071d9' is not in a state from which it can be stopped.

## To resolve the above wahala we need to use try and and except block( botocore)


In [91]:
def botocore_exceptions():
    """Using resource object to stop all instances. 
    Parameter: None
    Return: None
    """
    try:
        client = boto3.session.Session(profile_name="shared").client(service_name='ec2', region_name='us-east-1')
        waiter = client.get_waiter('instance_stopped')
        resource = boto3.session.Session(profile_name="shared").resource(service_name='ec2', region_name='us-east-1')
        print("stopping all instances....")
        resource.instances.stop()
        waiter.wait(InstanceIds=ids)
        print("all instances stopped!!...")
    except botocore.exceptions.ClientError as err:
        if err.response['Error']['Code'] == 'IncorrectInstanceState':
            print("Ooops, could not be stopped at this time")
        else:
            print(err.response['Error']['Code'])
    except botocore.exceptions.UnknownServiceError as err:
        print("Sorry invalid service")
    except Exception as e:
        print(e)
  
botocore_exceptions()
    

stopping all instances....
Ooops, could not be stopped at this time
